# OpenStreetMap Data Case Study

Tianxing Zhai

# Map Area

District of Columbia, U.S.

https://download.geofabrik.de/north-america/us/district-of-columbia-latest.osm.bz2

https://download.geofabrik.de/

I selected this area beacuse that District of Columbia is the smallest state and the capital of America.

# Part 1. Data wrangling

1. Transformation of data, from xml to csv

The first thing to do is to parse the xml and transform useful data from xml to csv. In the pratice, the tutor cleaned the data before transformtion. It is time consuming because every cleaning step is a traversal of a large xml file. So I decide to transform data first, and clean the data using Pandas library.

The cell below is my code of transformtion:

In [1]:
"""

The process for this transformation is as follows:
- Use iterparse to iteratively step through each top level element in the XML
- Shape each element into several data structures using a custom function
- Write each data structure to the appropriate .csv files

The "node" field will hold a dictionary of the following top level node attributes:
- id
- version
- lat
- lon
- timestamp
- changeset

The "node_tags" field will hold a list of dictionaries, one per secondary tag. Secondary tags are
child tags of node which have the tag name/type: "tag". Each dictionary should have the following
fields from the secondary tag attributes:
- id: the top level node id attribute value
- key: the full tag "k" attribute value if no colon is present or the characters after the colon if one is.
- value: the tag "v" attribute value
- type: either the characters before the colon in the tag "k" value or "regular" if a colon
        is not present.

Additionally,

- if the tag "k" value contains a ":" the characters before the ":" will be set as the tag type
  and characters after the ":" will be set as the tag key
- if there are additional ":" in the "k" value they and they will be ignored and kept as part of
  the tag key. For example:

  <tag k="addr:street:name" v="Lincoln"/>
  will be turned into
  {'id': 12345, 'key': 'street:name', 'value': 'Lincoln', 'type': 'addr'}

- If a node has no secondary tags then the "node_tags" field will just contain an empty list.

The final return value for a "node" element will look something like:

{'node': {'id': 757860928,
          'version': '2',
          'lat': 41.9747374,
          'lon': -87.6920102,
          'timestamp': '2010-07-22T16:16:51Z',
      'changeset': 5288876},
 'node_tags': [{'id': 757860928,
                'key': 'amenity',
                'value': 'fast_food',
                'type': 'regular'},
               {'id': 757860928,
                'key': 'cuisine',
                'value': 'sausage',
                'type': 'regular'},
               {'id': 757860928,
                'key': 'name',
                'value': "Shelly's Tasty Freeze",
                'type': 'regular'}]}

### If the element top level tag is "way":
The dictionary will have the format {"way": ..., "way_tags": ..., "way_nodes": ...}

The "way" field should hold a dictionary of the following top level way attributes:
- id
- version
- timestamp
- changeset


The "way_tags" field will again hold a list of dictionaries, following the exact same rules as
for "node_tags".

Additionally, the dictionary should have a field "way_nodes". "way_nodes" will hold a list of
dictionaries, one for each nd child tag.  Each dictionary will have the fields:
- id: the top level element (way) id
- node_id: the ref attribute value of the nd tag
- position: the index starting at 0 of the nd tag i.e. what order the nd tag appears within
            the way element

The final return value for a "way" element will look something like:

{'way': {'id': 209809850,
         'version': '1',
         'timestamp': '2013-03-13T15:58:04Z',
         'changeset': 15353317},
 'way_nodes': [{'id': 209809850, 'node_id': 2199822281, 'position': 0},
               {'id': 209809850, 'node_id': 2199822390, 'position': 1},
               {'id': 209809850, 'node_id': 2199822392, 'position': 2},
               {'id': 209809850, 'node_id': 2199822369, 'position': 3},
               {'id': 209809850, 'node_id': 2199822370, 'position': 4},
               {'id': 209809850, 'node_id': 2199822284, 'position': 5},
               {'id': 209809850, 'node_id': 2199822281, 'position': 6}],
 'way_tags': [{'id': 209809850,
               'key': 'housenumber',
               'type': 'addr',
               'value': '1412'},
              {'id': 209809850,
               'key': 'street',
               'type': 'addr',
               'value': 'West Lexington St.'},
              {'id': 209809850,
               'key': 'street:name',
               'type': 'addr',
               'value': 'Lexington'},
              {'id': '209809850',
               'key': 'street:prefix',
               'type': 'addr',
               'value': 'West'},
              {'id': 209809850,
               'key': 'street:type',
               'type': 'addr',
               'value': 'Street'},
              {'id': 209809850,
               'key': 'building',
               'type': 'regular',
               'value': 'yes'},
              {'id': 209809850,
               'key': 'levels',
               'type': 'building',
               'value': '1'},
              {'id': 209809850,
               'key': 'building_id',
               'type': 'chicago',
               'value': '366409'}]}
"""

import csv
import codecs
import re
import xml.etree.cElementTree as ET

OSM_PATH = "district-of-columbia-latest.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"


NODE_FIELDS = ['id', 'lat', 'lon', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = [] # Handle secondary tags the same way for both node and way elements

    
    if element.tag == 'node':
        
        node_attribs = element.attrib
        node_attribs['id'] = int(node_attribs['id'])
        node_attribs['lat'] = float(node_attribs['lat'])
        node_attribs['lon'] = float(node_attribs['lon'])
        node_attribs['changeset'] = int(node_attribs['changeset'])
        
        for tag in element.iter("tag"):
            
            tag_attribs = {}
            
            tag_attribs['id'] =  node_attribs['id']
            tag_attribs['value'] = tag.attrib['v']
                
            if ':' in tag.attrib['k']:
                temp1 = re.split(':',tag.attrib['k'])
                if len(temp1) == 2:
                    tag_attribs['type'] = temp1[0]
                    tag_attribs['key'] = temp1[1]
                else:
                    tag_attribs['type'] = temp1[0]
                    tag_attribs['key'] = ':'.join(temp1[1:])
            else:
                tag_attribs['type'] = 'regular'
                tag_attribs['key'] = tag.attrib['k']
                
            tags.append(tag_attribs)
            
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        
        way_attribs = element.attrib
        way_attribs['id'] = int(way_attribs['id'])
        way_attribs['changeset'] = int(way_attribs['changeset'])
        i = 0
        
        for tag in element.iter("tag"):
            
            tag_attribs = {}
            
            tag_attribs['id'] =  way_attribs['id']
            tag_attribs['value'] = tag.attrib['v']
                
            if ':' in tag.attrib['k']:
                temp1 = re.split(':',tag.attrib['k'])
                if len(temp1) == 2:
                    tag_attribs['type'] = temp1[0]
                    tag_attribs['key'] = temp1[1]
                else:
                    tag_attribs['type'] = temp1[0]
                    tag_attribs['key'] = ':'.join(temp1[1:])
            else:
                tag_attribs['type'] = 'regular'
                tag_attribs['key'] = tag.attrib['k']
                
            tags.append(tag_attribs)
            
        for nd in element.iter("nd"):
                
            nd_attribs = {}
                
            nd_attribs['id'] =  way_attribs['id']
            nd_attribs['node_id'] = int(nd.attrib['ref'])
            nd_attribs['position'] = i
            i += 1
            way_nodes.append(nd_attribs)
        
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""
    osm_file = open(osm_file,'r',encoding='utf-8')
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


class MyDictWriter(csv.DictWriter, object):
    """Write the data to csv files (Becuase Python3 uses utf-8, conversion of unicode is no longer needed)"""
    def writerow(self, row):
        super(MyDictWriter, self).writerow({
            k: v for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w', encoding='utf-8') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w', encoding='utf-8') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w', encoding='utf-8') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w', encoding='utf-8') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w', encoding='utf-8') as way_tags_file:

        nodes_writer = MyDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = MyDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = MyDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = MyDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = MyDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

In [2]:
process_map(OSM_PATH)

2. Cleaning of data, from csv to pandas dataframe

In [3]:
import re
import pandas as pd

# open the data as pandas dataframe
nodes = pd.read_csv('nodes.csv')
nodes_tags = pd.read_csv('nodes_tags.csv')
ways = pd.read_csv('ways.csv')
ways_nodes = pd.read_csv('ways_nodes.csv')
ways_tags = pd.read_csv('ways_tags.csv')

    2.1. Dealing with problematic characters

We define [=\+/&<>;\'"\?%#$@\,\. \t\r\n] as problematic characters. Keys which contains these characters are probably problematic. I try to find these information using Pandas which is much faster than the iteration method.

In [4]:
def find_problemchars (x):
    x = str(x)
    problemchars = re.compile(r'.*[=\+/&<>;\'"\?%#$@\,\. \t\r\n].*')
    if re.match(problemchars, x):
        return True
    else:
        return False

In [5]:
problemchars_bool_ways = ways_tags['key'].apply(find_problemchars)
problemchars_bool_nodes = nodes_tags['key'].apply(find_problemchars)

In [6]:
nodes_tags[problemchars_bool_nodes]

,id,key,value,type


In [7]:
ways_tags[problemchars_bool_ways]

,id,key,value,type
424280,67069962,citybikes.com,4,regular


There are no problematic characters in keys of node tags. There is only one problematic character in keys of way tags. To decide how to deal with this key, we need to know all attributes of the way with id 67069962. 

In [8]:
ways_tags.loc[ways_tags['id'] == 67069962]

,id,key,value,type
424273,67069962,source,dcgis,regular
424274,67069962,dataset,buildings,regular
424275,67069962,building,yes,regular
424276,67069962,lot,0802,dcgis
424277,67069962,street,Columbia Road Northwest,addr
424278,67069962,gis_id,103264,dcgis
424279,67069962,square,2564,dcgis
424280,67069962,citybikes.com,4,regular
424281,67069962,housenumber,1772,addr
424282,67069962,captureyear,19990331,dcgis


As we can see, the way 67069962 seems describe a building. One of the key is 'citybikes.com', a website, and the value of that key is 4. It is hard to decide whether to delete it or not because we don't know the relationship between that building and 'citybikes.com'. To be cautious, I decide to keep that row.

    2.2 Dealing with street names

["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", "Trail", "Parkway", "Commons"] are commly used street names. We regard these as 'expected' words and try to find street names end with 'unexpected' words, which might be problematic.

The first thing to do is to find  'unexpected' end words in street names.

In [9]:
def audit_street (x):
    expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
                "Trail", "Parkway", "Commons"]
    x = str(x)
    x = x.split()[-1]
    if x in expected:
        return 'expected'
    else:
        return x
    
def run_audit (data, function):
    return (data.loc[(data['type'] == 'addr') & (data['key'] == 'street')]['value']
            .apply(function)
            .value_counts())

'Unexpected' end words in ways_tags:

In [10]:
run_audit (ways_tags, audit_street)

Northwest     56886
Northeast     34509
Southeast     23569
Southwest      2274
NW               48
SW               48
SE               46
expected         30
NE                9
road)             4
North             2
3456              2
Ave               1
Highway           1
Southeast\        1
Plaza             1
S.W.              1
850               1
Name: value, dtype: int64

'Unexpected' end words in ways_tags:

In [11]:
run_audit (nodes_tags, audit_street)

Northwest    8219
Southeast    5236
Northeast    4783
Southwest    1438
NW             51
SW             31
NE             31
SE             30
expected       15
Alley           2
BN              2
Floor           2
1               2
Hill            1
Mall            1
avenue          1
20019           1
George          1
northwest       1
300             1
West            1
328             1
n.w.            1
Bottom          1
N.W.            1
St.)            1
floor           1
Name: value, dtype: int64

There are only 45 'expected' words in the two data sets. Most of street names end with "Northwest", "Southeast", "Northeast", "Southwest" and their abbreviations. This seems the naming rule of Washington D.C.. Besides, few names end with "North", "West", "Alley" and "Mall". These names should also be considered as normal words.

I alter the code to recognize new expected normal end words and expand some abbreviations (like "SW" to "Southwest"). After that, find the full street names with unexpected end words.

In [12]:
def audit_street_new (x):
    
    # add "Northwest", "Southeast", "Northeast","Southwest","North", "Alley", "Mall", "West" to expected words
    expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
                "Trail", "Parkway", "Commons", "Northwest", "Southeast", "Northeast","Southwest",
                "North", "Alley", "Mall", "West"]
    # expand some abbreviations
    correction = {'NW': 'Northwest',
             'SW': 'Southwest',
             'NE': 'Northeast',
             'SE': 'Southeast',
             'Ave': 'Avenue',
             'Southeast\\': 'Southeast',
             'S.W.': 'Southwest',
             'N.W.': 'Northwest',
             'avenue': 'Avenue',
             'northwest': 'Northwest',
             'n.w.': 'Northwest'}
    
    t = str(x)
    t = t.split()[-1]
    
    if t in correction:
        t = correction[t]
    if t in expected:
        return 'normal'
    else:
        return x

In [13]:
run_audit (ways_tags, audit_street_new)

normal                                    117424
K Street NW (access road)                      4
3456                                           2
Connecticut Avenue Northwest Suite 850         1
Canal Center Plaza                             1
Lee Highway                                    1
Name: value, dtype: int64

In [14]:
run_audit (nodes_tags, audit_street_new)

normal                                        19842
North Capitol Street BN                           2
1                                                 2
Calvert Street Northwest 2nd Floor                1
M Street Northwest, Suite 328                     1
5601 E Capitol St SE, Washington, DC 20019        1
Wisconsin Ave (on Jenifer St.)                    1
17th St NW, 9th floor                             1
13rd Avenue Prince George                         1
Connecticut Ave NW 2nd Floor                      1
Capitol Hill                                      1
K Street NW Suite 300                             1
23rd St NW ,Foggy Bottom                          1
Name: value, dtype: int64

After the correction above, there are still some abnormal names which only appear once or twice. I have to fix them case by case using the code below:

In [15]:
def fix_street_name (x):
    
    # add "Northwest", "Southeast", "Northeast","Southwest","North", "Alley", "Mall", "West"to expected words
    expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
                "Trail", "Parkway", "Commons", "Northwest", "Southeast", "Northeast","Southwest",
                "North", "Alley", "Mall", "West"]
    # expand some abbreviations
    correction = {'NW': 'Northwest',
             'SW': 'Southwest',
             'NE': 'Northeast',
             'SE': 'Southeast',
             'Ave': 'Avenue',
             'Southeast\\': 'Southeast',
             'S.W.': 'Southwest',
             'N.W.': 'Northwest',
             'avenue': 'Avenue',
             'northwest': 'Northwest',
             'n.w.': 'Northwest',
             'St': 'Street'}
    
    x = str(x)
    
    if x.isdigit():
        print (x + ' is labeled as abnormal value')
        return 'Abnormal value' # fix wrong street name with only digit, label them as abnormal value
    else:
        if ',' in x:
            x = x.split(',')[0] # fix wrong street name with comma
        if '(' in x:
            x = x.split(' (')[0] # fix wrong street name with parenthesis
        
        tlist = x.split()
        t = tlist[-1]
        
        if t.isdigit():
            x = ' '.join(tlist[:-2]) # fix wrong street name like 'Connecticut Avenue Northwest Suite 850 '
            tlist = x.split()
            t = tlist[-1]
        if t == 'Floor':
            x = ' '.join(tlist[:-2]) # fix the wrong street name 'Connecticut Ave NW 2nd Floor'
            tlist = x.split()
            t = tlist[-1]
        if t in correction:
            t = correction[t] # fix wrong street name with abbreviation
            tlist[-1] = t
            x = ' '.join(tlist)
        if t in expected:
            return x
        else:
            print (x + ' is labeled as abnormal value')
            return 'Abnormal value' # fix wrong street name like 'Canal Center Plaza', label them as abnormal value

def update (data):
    street_new = (data.loc[(data['type'] == 'addr') & (data['key'] == 'street')]['value']
                .apply(fix_street_name))
    data.update(street_new)

In [16]:
update (nodes_tags)

1 is labeled as abnormal value
1 is labeled as abnormal value
North Capitol Street BN is labeled as abnormal value
North Capitol Street BN is labeled as abnormal value
Capitol Hill is labeled as abnormal value
13rd Avenue Prince George is labeled as abnormal value


In [17]:
update (ways_tags)

Canal Center Plaza is labeled as abnormal value
Lee Highway is labeled as abnormal value
3456 is labeled as abnormal value
3456 is labeled as abnormal value


After cleaning the street names, I run the audit program again to test the result:

In [18]:
run_audit (nodes_tags, audit_street)

Northwest    8273
Southeast    5266
Northeast    4814
Southwest    1468
expected       25
value           6
Alley           2
West            1
Mall            1
Name: value, dtype: int64

In [19]:
run_audit (ways_tags, audit_street)

Northwest    56939
Northeast    34518
Southeast    23615
Southwest     2322
expected        33
value            4
North            2
Name: value, dtype: int64

We can see that there are no unexpected names anymore. The cleaning step successfully finished.

    2.3 Dealing with postcodes

Standard postcodes of Washington D.C. should have five digits and in the range from 20001 to 20600. Here is how I audit postcodes and find abnormal vaues:

In [20]:
def audit_postcode (x):
    x = str(x)
    if (len(x) != 5) | (not x.isdigit()):
        return x
    elif int(x) not in range(20001, 20600):
        return x
    else:
        return 'normal'
    
def run_audit (data):
    return (data.loc[data['key'] == 'postcode']['value']
            .apply(audit_postcode)
            .value_counts())

In [21]:
run_audit (ways_tags)

normal        61991
20910             6
20912             1
20020-4706        1
20005-1015        1
22207             1
20005-7700        1
20016-2137        1
20005-1009        1
20005-1001        1
22209             1
20005-1019        1
22101             1
20005-1013        1
Name: value, dtype: int64

In [22]:
run_audit (nodes_tags)

normal        9424
DC 20002         1
20006-5346       1
20005-4111       1
2005             1
20743            1
20036-5305       1
2011             1
20005-1015       1
22314            1
20009-5540       1
20005-5702       1
Name: value, dtype: int64

There four types abnormal values: 1. Postcodes with '-' (like 20006-5346). The numbers after '-' make postcodes more precise, but also make them not fit the format of most postcodes. For these postcodes, I will only keep the first five digits. 2. Postcode with 'DC' (DC 20002). For this postcode, I will only keep the digits. 3. Abnormal postcodes like 2005 and 2011, which are meaningless. I will label them as 'Abnormal value'. 4. Postcodes out of range (20001-20600). These are all postcodes of nearby cities. For example, 20910 is the postcode of Silver Spring, a city borders on Washington on the north. It is acceptable that the postcode data of Washington have few values come from neighboring cities, beacuse there are no remarkable boundaries between them. Therefore, I will keep these values.

Below are my codes of fixing postcodes and updating the orginal data sets:

In [23]:
def fix_postcode (x):
    x = str(x)
    if '-' in x:
        x = x.split('-')[0] # fix postcode like '20005-1015'
    if 'DC' in x:
        x = x.split()[1] # fix the postcode 'DC 20002'
    if len(x) == 5:
        return x
    else:
        print (x + ' is labeled as abnormal value') # 2011 and 2005 will be labeled as 'Abnormal value'
        return 'Abnormal value' 

def run_fix (data):
    postcodes_new = data.loc[data['key'] == 'postcode']['value'].apply(fix_postcode)
    data.update(postcodes_new)

In [24]:
run_fix (ways_tags)

In [25]:
run_fix (nodes_tags)

2011 is labeled as abnormal value
2005 is labeled as abnormal value


3. Export of data, from pandas dataframe to SQL

After finishing all the cleaning steps, the next step is to export the cleaned data from pandas dataframe to SQL.

The first thing to do is to creat five SQL tables with correct data types and references:

In [26]:
import sqlite3
conn = sqlite3.connect('Washington.db')
c = conn.cursor()
c.execute('''    
    CREATE TABLE nodes (
    id INTEGER PRIMARY KEY NOT NULL,
    lat REAL,
    lon REAL,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT)
    ''')
c.execute('''    
    CREATE TABLE nodes_tags (
    id INTEGER,
    key TEXT,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES nodes(id))
    ''')
c.execute('''    
    CREATE TABLE ways (
    id INTEGER PRIMARY KEY NOT NULL,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT)
    ''')
c.execute('''    
    CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id))
    ''')
c.execute('''    
    CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id))
    ''')
conn.commit()
conn.close()

Then export data from pandas dataframes to corresponding SQL tables:

In [27]:
conn = sqlite3.connect('Washington.db')
for dataframe in ['nodes', 'nodes_tags', 'ways', 'ways_tags', 'ways_nodes']:
    locals()[dataframe].to_sql(dataframe, conn, if_exists = 'append', index = False)

# Part 2. The overview of data

1.The size of files 

district-of-columbia-latest.osm ........ 316 MB                                             
Washington.db ................................ 180 MB    
nodes.csv ........................................ 95.7 MB   
nodes_tags.csv ............................... 5.80 MB   
ways.csv ......................................... 6.97 MB   
ways_nodes.csv ............................. 45.9 MB   
ways_tags.csv ................................ 43.1 MB

2. The amount of nodes and ways

Here is the code to run a query and print results by row:

In [28]:
import sqlite3

conn = sqlite3.connect('Washington.db')
c = conn.cursor()

def run_query (query):
    result = c.execute(query)
    for row in result:
        print(row)

The amount of nodes:

In [29]:
run_query ('SELECT COUNT(*) FROM nodes')

(1699412,)


The amount of ways:

In [30]:
run_query ('SELECT COUNT(*) FROM ways')

(206289,)


3. How many nodes and ways have tags?

In [31]:
run_query ('SELECT COUNT(DISTINCT(id)) FROM nodes_tags') 

(41181,)


In [32]:
run_query ('SELECT COUNT(DISTINCT(id)) FROM ways_tags') 

(202363,)


There are 41181 nodes have tags, amounted to only 2% of whole nodes. While there are 202363 ways have tags, amounted to 98% of whole ways. That means we can get better information from ways, not nodes.

4. Which nodes/ways have most tags and how many tags do they have?

The TOP 10 ways which have most tags:

In [33]:
run_query ('SELECT id, COUNT(*) FROM ways_tags GROUP BY id ORDER BY count(*) DESC LIMIT 10') 

(48268463, 55)
(255331569, 55)
(48207262, 54)
(48207538, 54)
(48207335, 53)
(48207345, 53)
(48207367, 51)
(48268465, 51)
(48207321, 50)
(48207330, 50)


The TOP 10 nodes which have most tags:

In [34]:
run_query ('SELECT id, COUNT(*) FROM nodes_tags GROUP BY id ORDER BY count(*) DESC LIMIT 10') 

(158368533, 149)
(2507807432, 35)
(367142763, 26)
(1181385550, 24)
(1181385554, 24)
(1181385557, 24)
(1181385567, 24)
(1181385573, 24)
(1181385584, 24)
(1181385594, 24)


We can see that ways with most tags have around 50+ tags and nodes with most tags around 20+ tags. However, there is an outlier: node id 158368533 has 149 tags! I need to find why.

Print all tags that id 158368533 have:

In [35]:
run_query ('SELECT * FROM nodes_tags WHERE id = 158368533') 

(158368533, 'admin_level', '2', 'regular')
(158368533, 'alt_name', 'Washington DC', 'regular')
(158368533, 'alt_name_1', 'Washington, D.C.', 'regular')
(158368533, 'vi', 'Oa-sinh-tơn', 'alt_name')
(158368533, 'capital', 'yes', 'regular')
(158368533, 'population', '2010', 'census')
(158368533, 'ele', '7', 'regular')
(158368533, 'Class', 'Populated Place', 'gnis')
(158368533, 'County', 'District of Columbia', 'gnis')
(158368533, 'County_num', '001', 'gnis')
(158368533, 'id', '531871', 'gnis')
(158368533, 'ST_alpha', 'DC', 'gnis')
(158368533, 'ST_num', '11', 'gnis')
(158368533, 'int_name', 'Washington, D.C.', 'regular')
(158368533, 'continent', 'North America', 'is_in')
(158368533, 'country', 'United States', 'is_in')
(158368533, 'country_code', 'US', 'is_in')
(158368533, 'iso_3166_2', 'US-DC', 'is_in')
(158368533, 'name', 'Washington', 'regular')
(158368533, 'am', 'ዋሺንግተን ዲሲ', 'name')
(158368533, 'an', 'Washington', 'name')
(158368533, 'ang', 'Hƿæsingatūn', 'name')
(158368533, 'ar', 'واش

After seeing all the tags, I know that the reason why this node have so many tags is that these tags describe Washington D.C. itself and its names in many languages.

5. Which ways have most/least nodes and how many nodes do they have?

In [36]:
run_query ('SELECT COUNT(*), id FROM ways_nodes GROUP BY id ORDER BY COUNT(*) DESC LIMIT 10') 

(1860, 232054381)
(1759, 289486048)
(1657, 405489795)
(1563, 42003302)
(1463, 493749667)
(1436, 232272021)
(1247, 183205906)
(946, 368672044)
(912, 52302188)
(853, 518015144)


In [37]:
run_query ('SELECT COUNT(*), id FROM ways_nodes GROUP BY id ORDER BY COUNT(*) LIMIT 10') 

(2, 6051300)
(2, 6051307)
(2, 6051328)
(2, 6051335)
(2, 6051337)
(2, 6051376)
(2, 6051409)
(2, 6051435)
(2, 6051438)
(2, 6051544)


The ways with most nodes have about 1000+ nodes. Each way has at least 2 nodes

6. Which way is the 'longest' way?

A interesting question comes into my mind: Which way is the 'longest' way? The 'longest' I define here is that the longest distance between the start point and end point of the way. The way with most nodes is probably not the longest way because we don't know neither whether the way is closed nor the straightness of the way. 

I decided to calculate the distance using the formula below:

$$ \sqrt[]{(lon_{end} - lon_{start})^2+(lat_{end} - lat_{start})^2}$$ ('lon' is longitude and 'lat' is latitude)

But sqlite do not support the calculation of square root. Instead, I use the formula below, just for comparison. 

$$ (lon_{end} - lon_{start})^2+(lat_{end} - lat_{start})^2 $$

Here is my strategy of designing the query:

1. Select rows with position 0 (table2) and maximum position (table1) from ways_nodes and join them together on id.  
   
   
'SELECT node_id_max, node_id_min, table1.id AS id  
FROM   
(SELECT node_id AS node_id_max, MAX(position), id FROM ways_nodes GROUP BY id) AS table1   
JOIN   
(SELECT node_id AS node_id_min, id FROM ways_nodes WHERE position = 0) AS table2  
ON  
table1.id = table2.id'  

2. Find out open ways, not closed ways. Open ways may be roads, highways, and railways etc.. Closed ways usually are areas like parks, schools etc.. The start (nodes at position 0) and end nodes (nodes at maximum position) of open ways are different. Name the table with open ways as table3.
  
  
'(SELECT node_id_max, node_id_min, table1.id AS id  
FROM  
(<font color=blue>SELECT node_id AS node_id_max, MAX(position), id FROM ways_nodes GROUP BY id</font>) AS <font color=blue>table1</font>   
JOIN   
(<font color=green>SELECT node_id AS node_id_min, id FROM ways_nodes WHERE position = 0</font>) AS <font color=green>table2</font>  
ON  
table1.id = table2.id  
<font color=red>WHERE node_id_max != node_id_min</font>) AS table3'

3. Use JOIN function to link two tables: table3 and nodes. To make start/end longitude and latitude in the same line with the corresponding ways_id.  
  
SELECT lon_start, lat_start, lon AS lon_end, lat AS lat_end, table4.ways_id AS ways_id   
FROM  
nodes  
JOIN  
(<font color=red>SELECT nodes.id, lon AS lon_start, lat AS lat_start, table3.node_id_min, table3.node_id_max, table3.id AS   ways_id  
FROM  
nodes  
JOIN  
table3  
ON  
nodes.id = table3.node_id_min</font>) AS <font color=red>table4</font>  
ON
nodes.id = table4.node_id_max  

4. Calulate the distance using the formula above and order by it.   
  
SELECT  
(lon_end - lon_start)*(lon_end - lon_start)+(lat_end - lat_start)*(lat_end - lat_start) AS distance_square,  
ways_id  
FROM  
table4  
ORDER BY distance_square DESC  
LIMIT 10  

Here is my full query:

In [38]:
query = '''
SELECT
(lon_end - lon_start)*(lon_end - lon_start)+(lat_end - lat_start)*(lat_end - lat_start) AS distance_square,
ways_id
FROM
(SELECT lon_start, lat_start, lon AS lon_end, lat AS lat_end, table4.ways_id AS ways_id 
FROM
nodes
JOIN
(SELECT nodes.id, lon AS lon_start, lat AS lat_start, table3.node_id_min, table3.node_id_max, table3.id AS ways_id
FROM
nodes
JOIN
(SELECT node_id_max, node_id_min, table1.id AS id
FROM 
(SELECT node_id AS node_id_max, MAX(position), id FROM ways_nodes GROUP BY id) AS table1 
JOIN 
(SELECT node_id AS node_id_min, id FROM ways_nodes WHERE position = 0) AS table2
ON
table1.id = table2.id
WHERE node_id_max != node_id_min) AS table3
ON
nodes.id = table3.node_id_min) AS table4
ON
nodes.id = table4.node_id_max)
ORDER BY distance_square DESC
LIMIT 10 
'''

Here is the TOP 10 longest ways with their id:

In [39]:
run_query (query)

(0.08060945860713437, 454552060)
(0.07471173990690035, 518015144)
(0.07464709465039931, 454552058)
(0.05497556201770428, 454552038)
(0.05031405611139642, 73533738)
(0.04332474482018089, 454552046)
(0.021600147350659692, 87759965)
(0.015386612247890353, 368672044)
(0.012504105768499724, 283869089)
(0.011528489170249531, 59524015)


To find what these ways are, I run another query to search:

In [40]:
run_query ('''
SELECT 
ways_tags.*
FROM
ways_tags 
JOIN
(SELECT
(lon_end - lon_start)*(lon_end - lon_start)+(lat_end - lat_start)*(lat_end - lat_start) AS distance_square,
ways_id
FROM
(SELECT lon_start, lat_start, lon AS lon_end, lat AS lat_end, table4.ways_id AS ways_id 
FROM
nodes
JOIN
(SELECT nodes.id, lon AS lon_start, lat AS lat_start, table3.node_id_min, table3.node_id_max, table3.id AS ways_id
FROM
nodes
JOIN
(SELECT node_id_max, node_id_min, table1.id AS id
FROM 
(SELECT node_id AS node_id_max, MAX(position), id FROM ways_nodes GROUP BY id) AS table1 
JOIN 
(SELECT node_id AS node_id_min, id FROM ways_nodes WHERE position = 0) AS table2
ON
table1.id = table2.id
WHERE node_id_max != node_id_min) AS table3
ON
nodes.id = table3.node_id_min) AS table4
ON
nodes.id = table4.node_id_max)
ORDER BY distance_square DESC
LIMIT 10) AS table5
ON 
ways_tags.id = table5.ways_id
ORDER BY distance_square DESC
''')

(518015144, 'source', 'NPS; Park Service Map; USGS NM', 'regular')
(454552058, 'source', 'DCGIS; NPS; Park Service Map; USGS NM', 'regular')
(454552038, 'leisure', 'park', 'regular')
(454552038, 'source', 'DCGIS;NPS;Park Service Map;USGS NM', 'regular')
(73533738, 'electrified', 'contact_line', 'regular')
(73533738, 'frequency', '25', 'regular')
(73533738, 'gauge', '1435', 'regular')
(73533738, 'operator', 'Philadelphia, Baltimore and Washington Railroad', 'historic')
(73533738, 'owner', 'Pennsylvania Railroad', 'historic')
(73533738, 'name', 'Northeast Corridor', 'regular')
(73533738, 'operator', 'Amtrak', 'regular')
(73533738, 'railway', 'rail', 'regular')
(73533738, 'source', 'USGS Ortho', 'regular')
(73533738, 'usage', 'main', 'regular')
(73533738, 'voltage', '12000', 'regular')
(87759965, 'name', 'Lower Potomac River', 'regular')
(87759965, 'source', 'USGS Ortho', 'regular')
(87759965, 'waterway', 'river', 'regular')
(87759965, 'wikidata', 'Q179444', 'regular')
(368672044, 'leisur

Not all the TOP 10 longest ways have tags. For example, the most longest way (id 454552060) have no tags, neither do id 454552046. Some tags are meaningless. For example, id 518015144 and 454552058 each have only one tag: 'source', which makes people can't know what do these ways represent. Besides, some tags probably are wrong. For example, id 454552038 and 368672044 each represents a park, according to their tags. But all the ways list here are open ways, not closed ways, which means they cannot represent closed areas (According to https://wiki.openstreetmap.org/wiki/Way ). There are only 4 meaningful ways: id 73533738 is a railway line, id 87759965 is a river, id 283869089 is a ferry route, and id 59524015 is a subway line.

# Part 3. Suggestion for improvement

Put more effort in the data auditing!

I do suggest that the developer of OpenStreetMap hire more people to do data auditing because the data quality is not so good:  
  
  
First, the format of street names and postcodes are not normalized. For street names, abbreviations and full names are both used, for example, 'NE' and 'Northeast'. For postcodes, five-digit codes are mixed with nine-digit codes. This may not be a big problem for human users, but not good for large-scale data processing by machine. They should make a standard for name and normalize all data.    

Second, some attributes are wrong. For example, 'Canal Center Plaza' should not be a street name, '2005' should not be a postcode, and an open way should not represent an area. These wrong attributes will be misleading for map users.  
  
I know OpenStreetMap is free and non-profit, so it is very hard for them to hire more people to do auditing. But maybe they can learn from what reCAPTCHAs did: reCAPTCHAs replace previous random CAPTCHAs with pictures of illegible ancient books and let people all around the world do the artificial recognition of books while doing CAPTCHAs entering. So I suggest that OpenStreetMap can learn from that and make a rule: An user must audit some old data of an area before he submit his new data of the same area.